### Install the Indexify Extractor SDK, Langchain Retriever and the Indexify Client

In [ ]:
%%capture
!pip install indexify-extractor-sdk indexify-langchain indexify

### Start the Indexify Server

In [ ]:
!./indexify server -d

### Download an Embedding Extractor
On another terminal we'll download and start the embedding extractor which we will use to index text from the pdf document.

In [ ]:
!indexify-extractor download hub://embedding/minilm-l6
!indexify-extractor join-server minilm-l6.minilm_l6:MiniLML6Extractor

### Download an Chunking Extractor
On another terminal we'll download and start the chunking extractor that will create chunks from the text and embeddings.

In [ ]:
!indexify-extractor download hub://text/chunking
!indexify-extractor join-server chunking.chunk_extractor:ChunkExtractor

### Download the PDF Extractor
On another terminal we'll install the necessary dependencies and start the PDF extractor which we will use to get text, bytes or json out of PDF documents.

Install Poppler on your machine

In [ ]:
!sudo apt-get install -y poppler-utils

Download and start the PDF extractor

In [ ]:
!indexify-extractor download hub://pdf/pdf-extractor
!indexify-extractor join-server pdf-extractor.pdf_extractor:PDFExtractor

### Create Extraction Policies
Instantiate the Indexify Client

In [2]:
from indexify import IndexifyClient
client = IndexifyClient()

First, create a policy to get texts and contents out of the PDF.

In [27]:
client.add_extraction_policy(extractor='tensorlake/pdf-extractor', name="pdf-extraction")

Lastly, create chunks from the text and embeddings

In [29]:
client.add_extraction_policy(extractor='tensorlake/chunk-extractor', name="chunks", content_source="pdf-extraction", input_params={"chunk_size": 512, "overlap": 150})

In [30]:
client.add_extraction_policy(extractor='BAAI/bge-base-en', name="bge-embedding", content_source="chunks")

### Upload a PDF File

In [31]:
import requests
req = requests.get("https://arxiv.org/pdf/2310.16944.pdf")

with open('2310.16944.pdf','wb') as f:
    f.write(req.content)

In [32]:
client.upload_file(path="2310.16944.pdf")

### What is happening behind the scenes

Indexify is designed to seamlessly respond to ingestion events by assessing all existing policies and triggering the necessary extractors for extraction. Once the PDF extractor completes the process of extracting texts, bytes, and JSONs from the document, it automatically initiates the embedding extractor to chunk the content, extract embeddings, and populate an index.

With Indexify, you have the ability to upload hundreds of PDF files simultaneously, and the platform will efficiently handle the extraction and indexing of the contents without requiring manual intervention. To expedite the extraction process, you can deploy multiple instances of the extractors, and Indexify's built-in scheduler will transparently distribute the workload among them, ensuring optimal performance and efficiency.

### Perform RAG
Initialize the Langchain Retreiver.

In [33]:
from indexify_langchain import IndexifyRetriever
params = {"name": "bge-embedding.embedding", "top_k": 15}
retriever = IndexifyRetriever(client=client, params=params)

Now create a chain to prompt OpenAI with data retreived from Indexify to create a simple Q&A bot

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [35]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Now ask any question related to the ingested PDF document

In [39]:
chain.invoke("What are the results on MT-Bench?")
# Results on MT-Bench show that ZEPHYR-7B surpasses LLAMA 2-CHAT-70B, the best open-access RLHF-based model.

'The results on MT-Bench show that Zephyr 7B model achieved a score of 7.34 and a win percentage of 90.60.'

In [40]:
chain.invoke("what prompts are they using? Show all the prompts ")

'The prompts being used are:\n1. Prompt (turn 1): Imagine you are participating in a race with a group of people. If you have just overtaken the second person, what’s your current position? Where is the person you just overtook?\n2. Prompt (turn 2): If the “second person” is changed to “last person” in the above question, what would the answer be?'